# Scenario 2: A cross-functional team with one DS working on an ML model

MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local`

In [ ]:
import mlflow

mlflow.set_tracking_server("http://127.0.01:5000")